In [304]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn import metrics
from sklearn.svm import LinearSVC
import string

from mi_helper import *

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline

from numpy import array 

In [305]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
# data_test = pd.read_csv("test_set.csv", sep = "\t")
# data_test.drop("Unnamed: 0", axis = 1, inplace = True)
# dataset = pd.concat([data_test, dataset])
# dataset.dropna(inplace=True)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.drop(dataset[dataset.rating == 4].index, inplace=True)
print(dataset["feedback"].value_counts())
dataset.info()

1    2286
0     257
Name: feedback, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2543 entries, 0 to 3148
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating            2543 non-null   int64 
 1   date              2543 non-null   object
 2   variation         2543 non-null   object
 3   verified_reviews  2543 non-null   object
 4   feedback          2543 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 119.2+ KB


In [306]:
# levare i numeri
# disambiguare la lemmatizzazione con pos
tokenizer = nltk.tokenize.TweetTokenizer()
lemmatizer = nltk.WordNetLemmatizer()
tokenized_reviews = []
for review in dataset["verified_reviews"]: #pulisce le recensioni
    clean_text = ""
    tokens = tokenizer.tokenize(review)
    clean_tokens = [w.lower() for w in tokens if w not in string.punctuation and len(w)>2]
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in clean_tokens]
    for t in lemmatized_tokens:
        clean_text += " " + t
    tokenized_reviews.append(clean_text)

dataset["verified_reviews"] = tokenized_reviews

In [307]:
# CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
reviews_1.sort(key=len, reverse = True)
sample_1 = reviews_1[:len(reviews_0)]
verified_reviews_sample = []
feedback_sample = []
verified_reviews_sample.extend(sample_1)
verified_reviews_sample.extend(reviews_0)
feedback_sample.extend([1 for i in range(len(sample_1))])
feedback_sample.extend([0 for i in range(len(reviews_0))])


dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
dataset["feedback"].value_counts()

1    257
0    257
Name: feedback, dtype: int64

In [308]:
tokenizer = nltk.tokenize.TweetTokenizer()
cv = CountVectorizer(stop_words="english", ngram_range=(1, 3), tokenizer=tokenizer.tokenize, min_df = 2) #count == frequenza
text_counts = cv.fit_transform(dataset["verified_reviews"]) 
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset["feedback"], test_size = 0.20, random_state=10) #divisione int raine test

In [309]:
text_counts

<514x5769 sparse matrix of type '<class 'numpy.int64'>'
	with 21229 stored elements in Compressed Sparse Row format>

In [310]:
len(cv.vocabulary_)

5769

In [311]:
cv.vocabulary_ # A mapping of terms to feature indices.

{'piece': 3636,
 'technology': 4896,
 'right': 4214,
 'center': 801,
 'living': 2958,
 'room': 4230,
 'island': 2561,
 'kitchen': 2707,
 'counter': 1060,
 'mic': 3142,
 'speaker': 4641,
 'direction': 1310,
 'quality': 3956,
 'sound': 4581,
 'quite': 4010,
 'good': 2007,
 'connected': 976,
 'echo': 1518,
 'bluetooth': 613,
 'soundbar': 4628,
 '360': 36,
 "it's": 2577,
 'audiophile': 458,
 'equipment': 1674,
 'range': 4037,
 'decent': 1164,
 'bass': 515,
 'indoor': 2495,
 'entertaining': 1668,
 'loud': 3015,
 'bother': 651,
 'building': 693,
 'work': 5663,
 'great': 2074,
 'volume': 5365,
 'device': 1198,
 'imagine': 2462,
 'having': 2221,
 'button': 710,
 'large': 2751,
 'recommend': 4122,
 'regular': 4137,
 '2nd': 28,
 'generation': 1950,
 'look': 2986,
 'weight': 5594,
 'material': 3126,
 'ha': 2128,
 'order': 3490,
 'came': 766,
 'free': 1874,
 'philip': 3605,
 'hue': 2404,
 'bulb': 700,
 'installed': 2520,
 'extra': 1723,
 'bought': 655,
 'floor': 1852,
 'lamp': 2740,
 'turned': 518

In [312]:
#get_feature_names_out([input_features])
#Get output feature names for transformation.

len(cv.get_feature_names())

5769

In [313]:
X_train[0,:]

<1x5769 sparse matrix of type '<class 'numpy.int64'>'
	with 43 stored elements in Compressed Sparse Row format>

In [314]:
print(X_train[0,:])

  (0, 4896)	1
  (0, 4641)	1
  (0, 1518)	1
  (0, 2986)	1
  (0, 5372)	1
  (0, 162)	1
  (0, 3741)	1
  (0, 3021)	2
  (0, 2350)	1
  (0, 2298)	1
  (0, 5555)	1
  (0, 4712)	1
  (0, 1508)	1
  (0, 2424)	1
  (0, 4509)	2
  (0, 2315)	1
  (0, 4526)	1
  (0, 4840)	1
  (0, 5248)	2
  (0, 4365)	1
  (0, 1015)	1
  (0, 139)	1
  (0, 5510)	1
  (0, 3385)	1
  (0, 2836)	1
  (0, 383)	1
  (0, 384)	1
  (0, 4489)	1
  (0, 3077)	1
  (0, 5614)	1
  (0, 868)	1
  (0, 1588)	1
  (0, 142)	1
  (0, 3417)	1
  (0, 3746)	1
  (0, 4528)	1
  (0, 4308)	1
  (0, 2967)	1
  (0, 4819)	1
  (0, 5515)	1
  (0, 5431)	1
  (0, 4392)	1
  (0, 250)	1


In [315]:
cv.inverse_transform(X_train[0,:]) #0 ==> prima recensione

[array(['technology', 'speaker', 'echo', 'look', 'wa', 'alexa', 'plug',
        'love', 'hooked', 'home', 'weather', 'spot', 'easy', "i'm",
        'smart', "home i'm", 'smart plug', 'sure', 'use', 'set', 'control',
        'alarm', "wasn't", 'news', 'lighting', 'ask', 'ask alexa', 'sleek',
        'love sleek', 'wi-fi', 'clock', 'echo spot', 'alarm clock',
        'nightstand', 'plug control', 'smart plug control', 'scroll',
        'local', 'suggestion', "wasn't sure", 'wa set', 'set use',
        'alexa use'], dtype='<U34')]

In [316]:
for feat,freq in zip(cv.inverse_transform(X_train[0,:])[0],X_train[0,:].data):
    print(feat,freq)

technology 1
speaker 1
echo 1
look 1
wa 1
alexa 1
plug 1
love 2
hooked 1
home 1
weather 1
spot 1
easy 1
i'm 1
smart 2
home i'm 1
smart plug 1
sure 1
use 2
set 1
control 1
alarm 1
wasn't 1
news 1
lighting 1
ask 1
ask alexa 1
sleek 1
love sleek 1
wi-fi 1
clock 1
echo spot 1
alarm clock 1
nightstand 1
plug control 1
smart plug control 1
scroll 1
local 1
suggestion 1
wasn't sure 1
wa set 1
set use 1
alexa use 1


In [317]:
#For classification we'll set 'chi2'  method as a scoring function. The target number of features is defined by k parameter
select = SelectKBest(chi2, k=5000)  # feature selection
select.fit(X_train,Y_train)

X_train_sel = select.transform(X_train)
X_test_sel = select.transform(X_test)

In [318]:
#We've selected 3 best features in x data. To identify the selected features we use get_support() function and filter out them from the features name list.  The z object contains selected x data
filter = select.get_support() #filtra le parole selezionando quelle contenenti nei k 5000
sum(filter)

5000

In [319]:
X_train_sel

<411x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 14302 stored elements in Compressed Sparse Row format>

In [320]:
print(X_test_sel[0,:])

  (0, 1738)	1
  (0, 1492)	1
  (0, 1141)	1
  (0, 1942)	1
  (0, 3867)	1
  (0, 3875)	1
  (0, 2993)	1
  (0, 3325)	1
  (0, 4584)	1
  (0, 4509)	1
  (0, 4513)	1
  (0, 3347)	1


In [321]:
print(cv.inverse_transform(select.inverse_transform(X_train_sel[0,:]))) #applichi l'inverse_transform sui vettori che poi hai utilizzato con il Select

[array(['alarm', 'alarm clock', 'alexa', 'alexa use', 'ask', 'ask alexa',
       'clock', 'control', 'easy', 'echo', 'echo spot', 'home',
       "home i'm", 'hooked', "i'm", 'lighting', 'look', 'love',
       'love sleek', 'news', 'nightstand', 'plug', 'plug control', 'set',
       'sleek', 'smart', 'smart plug', 'smart plug control', 'speaker',
       'spot', 'suggestion', 'sure', 'technology', 'use', 'wa', "wasn't",
       "wasn't sure", 'weather'], dtype='<U34')]


In [322]:
print(X_train_sel[0,:])

  (0, 4219)	1
  (0, 3989)	1
  (0, 1254)	1
  (0, 2539)	1
  (0, 4647)	1
  (0, 102)	1
  (0, 3200)	1
  (0, 2573)	2
  (0, 1983)	1
  (0, 1942)	1
  (0, 4813)	1
  (0, 4057)	1
  (0, 1244)	1
  (0, 2045)	1
  (0, 3867)	2
  (0, 1957)	1
  (0, 3882)	1
  (0, 4168)	1
  (0, 4535)	2
  (0, 3742)	1
  (0, 824)	1
  (0, 86)	1
  (0, 4770)	1
  (0, 2886)	1
  (0, 2410)	1
  (0, 286)	1
  (0, 287)	1
  (0, 3848)	1
  (0, 2621)	1
  (0, 703)	1
  (0, 1316)	1
  (0, 87)	1
  (0, 2916)	1
  (0, 3205)	1
  (0, 3884)	1
  (0, 4147)	1
  (0, 4775)	1
  (0, 169)	1


In [323]:
tfidf = TfidfTransformer()  # weighting
tfidf.fit(X_train_sel)
X_train_vec = tfidf.transform(X_train_sel)
X_test_vec =tfidf.transform(X_test_sel)

In [324]:
print(X_train_vec[0,:])

  (0, 4813)	0.11206031438287523
  (0, 4775)	0.20930093077870954
  (0, 4770)	0.16675193747248032
  (0, 4647)	0.08233188116876823
  (0, 4535)	0.19801389194661803
  (0, 4219)	0.16675193747248032
  (0, 4168)	0.13731651786073557
  (0, 4147)	0.22363026787736826
  (0, 4057)	0.1459792665445741
  (0, 3989)	0.09849101719433849
  (0, 3884)	0.19124810104021867
  (0, 3882)	0.15747985483696678
  (0, 3867)	0.23377392243463502
  (0, 3848)	0.20930093077870954
  (0, 3742)	0.10987360706759237
  (0, 3205)	0.19124810104021867
  (0, 3200)	0.14406850425189646
  (0, 2916)	0.19124810104021867
  (0, 2886)	0.15486084251394983
  (0, 2621)	0.20930093077870954
  (0, 2573)	0.1693552177493006
  (0, 2539)	0.1524226003738216
  (0, 2410)	0.19124810104021867
  (0, 2045)	0.11359709970742454
  (0, 1983)	0.17935700608168817
  (0, 1957)	0.22363026787736826
  (0, 1942)	0.11359709970742454
  (0, 1316)	0.16675193747248032
  (0, 1254)	0.0727370520554217
  (0, 1244)	0.13436977063533073
  (0, 824)	0.1405315054152911
  (0, 703)	0.1

In [325]:
cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))

[array(['alarm', 'alarm clock', 'alexa', 'alexa use', 'ask', 'ask alexa',
        'clock', 'control', 'easy', 'echo', 'echo spot', 'home',
        "home i'm", 'hooked', "i'm", 'lighting', 'look', 'love',
        'love sleek', 'news', 'nightstand', 'plug', 'plug control', 'set',
        'sleek', 'smart', 'smart plug', 'smart plug control', 'speaker',
        'spot', 'suggestion', 'sure', 'technology', 'use', 'wa', "wasn't",
        "wasn't sure", 'weather'], dtype='<U34')]

In [326]:
for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))[0],X_train_vec[0,:].data,X_train_sel[0,:].data):
    print(feat,round(weight,4),freq)

alarm 0.1121 1
alarm clock 0.2093 1
alexa 0.1668 1
alexa use 0.0823 1
ask 0.198 1
ask alexa 0.1668 1
clock 0.1373 1
control 0.2236 1
easy 0.146 1
echo 0.0985 1
echo spot 0.1912 1
home 0.1575 1
home i'm 0.2338 1
hooked 0.2093 1
i'm 0.1099 1
lighting 0.1912 1
look 0.1441 1
love 0.1912 2
love sleek 0.1549 1
news 0.2093 1
nightstand 0.1694 1
plug 0.1524 1
plug control 0.1912 1
set 0.1136 1
sleek 0.1794 1
smart 0.2236 2
smart plug 0.1136 1
smart plug control 0.1668 1
speaker 0.0727 1
spot 0.1344 1
suggestion 0.1405 1
sure 0.1279 1
technology 0.1705 1
use 0.1279 2
wa 0.1912 1
wasn't 0.0756 1
wasn't sure 0.1524 1
weather 0.1178 1


In [327]:
svm = LinearSVC()  # linear svm with default parameters
svm_clf = svm.fit(X_train_vec,Y_train)
predictions = svm_clf.predict(X_test_vec)

In [328]:
len(predictions), sum(predictions)

(103, 45)

In [329]:
predictions

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [330]:
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89        56
           1       0.89      0.85      0.87        47

    accuracy                           0.88       103
   macro avg       0.88      0.88      0.88       103
weighted avg       0.88      0.88      0.88       103



In [331]:
feature_names = cv.get_feature_names()
feature_names

['...',
 '... alexa',
 '... alexa play',
 '... amazing',
 '... amazing device',
 '... amazon',
 '... ha',
 '... ha bluetooth',
 '... item',
 '... item named',
 '... just',
 '... list',
 '... list definitely',
 '... lol',
 '... love',
 '... love listening',
 '... named',
 '... named computer',
 '... prime',
 '... set',
 '... set timer',
 '... sure',
 "... sure i'll",
 '... wa',
 '100',
 '100 amazon',
 '100 amazon music',
 '1st',
 '2nd',
 '2nd echo',
 '2nd echo dot',
 '2nd gen',
 '2nd gen echo',
 '2nd gen prime',
 '2nd generation',
 '2nd generation best',
 '360',
 '3rd',
 '3rd echo',
 '3rd echo product',
 '3rd party',
 '4am',
 '4am base',
 '4am base metro',
 '5th',
 'ability',
 'ability play',
 'able',
 'able download',
 'able download installing',
 'able hook',
 'able hook thermostat',
 'able install',
 'able install alexa',
 'able play',
 'able play music',
 'able set',
 'able set default',
 'able start',
 'able start morning',
 'able tell',
 'able voice',
 'able voice control',
 'abso

In [332]:
print(select.scores_)

[0.08267931 1.91428571 1.91428571 ... 2.08955224 2.08955224 4.17910448]


In [333]:
feature_names = cv.get_feature_names()
feats_w_score = list()
for index,(selected,score) in enumerate(zip(filter,select.scores_)):
    feats_w_score.append((score,selected,feature_names[index]))
feats_w_score = sorted(feats_w_score, reverse = True)
len(feats_w_score)

5769

In [334]:
feats_w_score[:10]

[(nan, False, '... amazing'),
 (nan, False, '... amazing device'),
 (83.31236784250473, True, 'alexa'),
 (23.67479373319736, True, 'alarm'),
 (19.142857142857142, True, 'house'),
 (14.357142857142854, True, 'amazing'),
 (8.614285714285714, True, 'alexa play'),
 (6.714833993298811, True, 'amazon music'),
 (6.699999999999999, True, 'assistant'),
 (6.699999999999999, True, 'alexa device')]

In [335]:
type(svm)

sklearn.svm._classes.LinearSVC

In [336]:
svm.coef_

array([[ 0.13465606,  0.13465606, -0.12849991, ..., -0.15674181,
        -0.30525992, -0.19055611]])

In [337]:
feats_w_classifier_weight = list()
for index,weight in enumerate(select.inverse_transform(svm.coef_)[0]):
    if weight!=0:
        feats_w_classifier_weight.append((weight,feature_names[index]))
feats_w_classifier_weight = sorted(feats_w_classifier_weight)
len(feats_w_classifier_weight)

4980

In [338]:
feats_w_classifier_weight[-100:] #features positive

[(0.3848130651300787, 'enjoying'),
 (0.3849658712990676, 'plane'),
 (0.3849658712990676, 'toy'),
 (0.3862676432940094, 'best'),
 (0.38660177547612967, 'amazon music'),
 (0.38850513483825566, 'home'),
 (0.3889862019734255, 'super'),
 (0.3905345640574321, 'timer'),
 (0.39170870836279814, 'smart home'),
 (0.39176616870559106, 'utilize'),
 (0.3977688625880097, 'yard'),
 (0.4001631479237308, 'turn light'),
 (0.4026630002587348, 'really'),
 (0.4064015987575404, 'way better'),
 (0.406779808970871, 'amazed'),
 (0.40989026533724054, 'spotify'),
 (0.4115290915844757, 'office'),
 (0.41336780923036764, "i'm"),
 (0.41382690557612334, 'prime day'),
 (0.41765825273305013, 'alexa getting'),
 (0.422081772374239, 'turn screen'),
 (0.42238461865478505, 'breeze'),
 (0.424876634714423, 'bedroom'),
 (0.4281609700636522, 'alexa device'),
 (0.4300674660796342, 'wake'),
 (0.43022233748734734, 'download'),
 (0.43481778318666625, 'wireless'),
 (0.43510194630975546, 'recommend'),
 (0.43671820093351166, 'forward')

In [339]:
feats_w_classifier_weight[:100] #features negative

[(-1.0465681042654538, 'return'),
 (-0.8930786138199194, 'money'),
 (-0.7704863760444365, 'disappointed'),
 (-0.750947808739038, 'fix'),
 (-0.7029584153061263, 'sonos'),
 (-0.6729726192497321, 'terrible'),
 (-0.6538068963298649, 'dot refurbished'),
 (-0.6218719820308805, 'poor'),
 (-0.5790394736435311, 'unable'),
 (-0.5661005850715269, 'sound terrible'),
 (-0.5512572995911736, 'tried'),
 (-0.5505618142501608, 'sound like'),
 (-0.5401121888561494, 'awful'),
 (-0.5351936398153481, 'sent'),
 (-0.5200965094769878, 'product wa'),
 (-0.512994427085878, 'firmware'),
 (-0.4968737088226379, 'half'),
 (-0.4937407070756744, 'different echo'),
 (-0.4922867671869764, 'month'),
 (-0.4863124740999543, 'refurbished'),
 (-0.4855201693231975, 'stop'),
 (-0.47046136577788633, 'fan'),
 (-0.46649770679831604, 'send'),
 (-0.4660132586145505, 'waste'),
 (-0.4606661571640104, 'support'),
 (-0.45416017985870377, "won't"),
 (-0.4484863197192018, 'seeing'),
 (-0.4477846319954123, 'useless'),
 (-0.445926422461057

In [340]:
stringa = """I hate this little device from hell. I used to love it. I bought one for every room in my house, my office at work, and they were good. But now, despite Amazon's alleged spying, some genius at Amazon decided to allow this thing to wake you up all hours of the night with a loud obnoxious noise and a bright yellow light. It will say you have a notification. Oh good, it must be a really good reason for Amazon to notify me of something mid sleep at 2 am on a Tuesday you ponder so you wake up the rest of the house by saying Alexa play notifications. Here's the best part, this little Alien Intelligence Robot without feet from hell will say Please rate the pair of shorts your wife bought last week, we need your review. Well I sure do love reviews, but at 2 am after being woken up to leave one, sadly I don't give a flying poopsicle about the shorts and a bug 1 star rating is all the revenge I can muster. This has been going on for weeks now. And it's enough to drive you mad because, you will contemplate unplugging the dirty little money siphon and cutting your losses, but you'll give the demon one more chance and she will allow you to sleep for a length of time in which you will forgive her treachery but then Boom, like a theif in the night a Hurricane will be boiling out in the Atlantic and this little weather girl from hell will want to notify you of the eminent doom 3 days ahead when the storm will be in your state. You will think how nice it is for all of the warnings you have heard thus far in social media,, television, and every news outlet in existence to be repeated to you as you climb into bed, but it gets even better. As soon as you listen to this golden nugget of doomsday propaganda and ponder how it is that you will sleep now because 3 days may not be enough time to dig a deep enough hole to hide from impending doom, she lights up again, oh my maybe flying dragons are headed this way, I better play the new notification, when your wife wakes up and says stop she has been doing that all evening, they literally keep changing the time of the storm by maybe an hour this way or that. Well by all means, the information could save one of our spoiled lives and we wouldn't want anyone to not know at which precise moment the raindrops will occur, but I do miss life before the artificial noises and alerts, bright yellow lights and bad news desserts. You may say that this is a growing pain with Alexa and AI technology kinks in the programming. I say with the amount of money I have poured into this monopolistic conglomerate, I should at least expect a decent night's sleep in return. You may say in page 74 paragraph D of the updated and revised owner's pamphlet there is a way to turn this "Feature" off, I welcome that comment but feel the need to say that I am not looking to become even slightly better educated on the device, I simply bought it to use as a voice controlled radio, this extra bologna does not appeal to me and has made me regret this purchase and every other from the brown box hq who secretly reads my mind, except when it comes to allowing me sleep, although kudos for recommending the sleep sounds, but it rarely works. We just need to be mindful of the 6 hours that most people like to not be disturbed. I regret selling my Amazon stock at 2400 and I regret purchasing all of these Alexa devices, and don't get me started on teenagers and Alexa devices, the combo that don't mix. They always, always, always leave the device at top volume, so when you walk into the bathroom at dawn and whisper Alexa what time is it, you then need to set an appointment for a blown eardrum and apologize to everyone for waking them up while the teen sleeps soundly through it all. Congratulations Amazon, you have destroyed my sanity. Weird goal but excellent execution."""

In [341]:
vector = cv.transform([stringa])
vector = select.transform(vector)
vector = tfidf.transform(vector)
predicted = svm.predict(vector)
print(predicted)

[1]


In [342]:
MNB = MultinomialNB()  # linear svm with default parameters
MNB_clsf = MNB.fit(X_train_vec,Y_train)
predictions = MNB_clsf.predict(X_test_vec)
print(metrics.classification_report(predictions, Y_test))

              precision    recall  f1-score   support

           0       0.64      0.90      0.75        40
           1       0.91      0.68      0.78        63

    accuracy                           0.77       103
   macro avg       0.78      0.79      0.77       103
weighted avg       0.81      0.77      0.77       103

